In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
covid = pd.read_csv('http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip')
coords = pd.read_csv('coordenadas.csv')

In [3]:
yuc_coords = coords[coords['Num_Ent'] == 31]
yuc_coords

,Entidad,Num_Ent,Municipio,Num_Mun,lat,lon
2301,Yucatán,31,Abalá,1,20.647578,-89.680886
2302,Yucatán,31,Acanceh,2,20.812422,-89.453547
2303,Yucatán,31,Akil,3,20.267053,-89.346826
2304,Yucatán,31,Baca,4,21.109548,-89.399382
2305,Yucatán,31,Bokobá,5,21.006940,-89.179525
...,...,...,...,...,...,...
2402,Yucatán,31,Valladolid,102,20.689726,-88.200980
2403,Yucatán,31,Xocchel,103,20.833489,-89.183719
2404,Yucatán,31,Yaxcabá,104,20.549126,-88.827206
2405,Yucatán,31,Yaxkukul,105,21.063479,-89.419238


In [4]:
d = {n:m for n, m in zip(yuc_coords.Num_Mun, yuc_coords.Municipio)}
numb = [1,2,3]
diag = ['Positivo', 'Negativo', 'Por confirmar']
d2 = {n2:m2 for n2, m2 in zip(numb, diag)}

In [5]:
yucatan = covid[covid['ENTIDAD_RES'] == 31]

In [6]:
yuc2 = yucatan.groupby(by=['MUNICIPIO_RES'])['RESULTADO'].value_counts().rename(columns={'RESULTADO':'SUMA'}).reset_index().rename(columns={0:'SUMA'})
yuc2['MUNICIPIO'] = [d.get(m) for m in yuc2['MUNICIPIO_RES'].values.tolist()]
yuc2['RESULTADO2'] = [d2.get(m) for m in yuc2['RESULTADO'].values.tolist()]
#yuc2

In [7]:
muni = np.unique(yuc2['MUNICIPIO'])
pos = []
neg = []
conf = []
for i in muni:
    prov1 = yuc2[(yuc2['MUNICIPIO'] == i) & (yuc2['RESULTADO2'] == 'Positivo')]
    prov2 = yuc2[(yuc2['MUNICIPIO'] == i) & (yuc2['RESULTADO2'] == 'Negativo')]
    prov3 = yuc2[(yuc2['MUNICIPIO'] == i) & (yuc2['RESULTADO2'] == 'Por confirmar')]
    if len(prov1['SUMA'].values.tolist()) == 0:
        pos.append(0)
    else:
        pos.append(prov1['SUMA'].iloc[0])
    if len(prov2['SUMA'].values.tolist()) == 0:
        neg.append(0)
    else:
        neg.append(prov2['SUMA'].iloc[0])
    if len(prov3['SUMA'].values.tolist()) == 0:
        conf.append(0)
    else:
        conf.append(prov3['SUMA'].iloc[0])

In [8]:
#list(zip(pos, neg, conf))

In [9]:
lat = {v:t for v, t in zip(yuc_coords.Municipio, yuc_coords.lat)}
lon = {v:t for v, t in zip(yuc_coords.Municipio, yuc_coords.lon)}
lat_muni = [lat.get(v) for v in muni]
lon_muni = [lon.get(v) for v in muni]
#dataf = 

In [13]:
data_f = pd.DataFrame()
data_f['Municipio'] = muni
data_f['Positivos'] = pos
data_f['Negativos'] = neg
data_f['Por confirmar'] = conf
data_f['Latitud'] = lat_muni
data_f['Longitud'] = lon_muni
data_f['Tamaño'] = np.asarray(pos) + 100

In [55]:
import plotly.express as px

fig = px.scatter_mapbox(data_f, lat="Latitud", lon="Longitud", hover_name="Municipio",
                        color_discrete_sequence=["red"], zoom=8, height=500, size='Tamaño', hover_data=["Positivos", "Negativos", "Por confirmar"],
                       center={'lat':lat.get('Mérida'), 'lon':lon.get('Mérida')})
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [38]:
sexo_dict = {1:'Mujer', 2:'Hombre', 3:'No especificado'}
#sex = [sexo_dict.get(v) for v in yucatan['SEXO'].values.tolist()]
yucatan.columns.values.tolist()

['FECHA_ACTUALIZACION',
 'ID_REGISTRO',
 'ORIGEN',
 'SECTOR',
 'ENTIDAD_UM',
 'SEXO',
 'ENTIDAD_NAC',
 'ENTIDAD_RES',
 'MUNICIPIO_RES',
 'TIPO_PACIENTE',
 'FECHA_INGRESO',
 'FECHA_SINTOMAS',
 'FECHA_DEF',
 'INTUBADO',
 'NEUMONIA',
 'EDAD',
 'NACIONALIDAD',
 'EMBARAZO',
 'HABLA_LENGUA_INDIG',
 'DIABETES',
 'EPOC',
 'ASMA',
 'INMUSUPR',
 'HIPERTENSION',
 'OTRA_COM',
 'CARDIOVASCULAR',
 'OBESIDAD',
 'RENAL_CRONICA',
 'TABAQUISMO',
 'OTRO_CASO',
 'RESULTADO',
 'MIGRANTE',
 'PAIS_NACIONALIDAD',
 'PAIS_ORIGEN',
 'UCI']

In [48]:
cols = ['NEUMONIA', 'DIABETES', 'EPOC', 'INMUSUPR', 'ASMA', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
        'RENAL_CRONICA', 'OTRA_COM', 'OTRO_CASO', 'SEXO', 'RESULTADO']
#yucatan[cols]
yuc3 = yucatan[cols]
sex = [sexo_dict.get(v) for v in yuc3['SEXO'].values.tolist()]
yuc3['Gender'] = sex

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
data3 = yuc3[yuc3['RESULTADO'] == 1]
neumonia = data3[data3['NEUMONIA'] == 1]
diabetes = data3[data3['DIABETES'] == 1]
epoc = data3[data3['EPOC'] == 1]
hiper = data3[data3['HIPERTENSION'] == 1]
ob = data3[data3['OBESIDAD'] == 1]
car = data3[data3['CARDIOVASCULAR'] == 1]

In [57]:
neumonia

,NEUMONIA,DIABETES,EPOC,INMUSUPR,ASMA,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,OTRA_COM,OTRO_CASO,SEXO,RESULTADO,Gender
11,1,2,2,2,2,1,2,2,2,1,2,2,1,Hombre
50,1,2,2,2,2,2,2,1,2,2,1,2,1,Hombre
2384,1,1,2,2,2,1,2,1,2,2,99,2,1,Hombre
6690,1,2,2,2,2,2,2,2,2,2,2,2,1,Hombre
8546,1,2,2,2,2,1,2,2,2,2,1,2,1,Hombre
9017,1,2,2,2,2,2,2,1,2,2,1,2,1,Hombre
9885,1,2,2,2,2,1,2,1,2,2,99,2,1,Hombre
10180,1,2,2,2,2,2,1,2,2,2,99,2,1,Hombre
12148,1,2,2,2,2,2,2,2,2,2,1,2,1,Hombre
12223,1,1,2,2,2,2,2,1,2,2,99,1,1,Mujer


In [103]:
import plotly.express as px
fig2 = go.Figure()
fig2.add_trace(go.Histogram(histfunc="count", y=neumonia['NEUMONIA'], x=neumonia['Gender'], name="NEUMONIA"))
fig2.add_trace(go.Histogram(histfunc="count", y=diabetes['DIABETES'], x=diabetes['Gender'], name="DIABETES"))
fig2.add_trace(go.Histogram(histfunc="count", y=epoc['EPOC'], x=epoc['Gender'], name="EPOC"))
fig2.add_trace(go.Histogram(histfunc="count", y=hiper['HIPERTENSION'], x=hiper['Gender'], name="HIPERTENSION"))
fig2.add_trace(go.Histogram(histfunc="count", y=ob['OBESIDAD'], x=ob['Gender'], name="OBESIDAD"))
fig2.add_trace(go.Histogram(histfunc="count", y=car['CARDIOVASCULAR'], x=car['Gender'], name="CARDIOVASCULAR"))
fig2.update_layout(title="Número de casos por enfermedad y sexo",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"), title_x=0.5)
fig2.show()